# Lab6.2: Topic modeling using gensim

In this notebook, we demonstrate how LDA models can be built and applied using the *gensim* package.

Credits:

This notebook is an adaptation of a blog from Susan Li's:

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24



The data set we’ll use is a list of over one million news headlines published over a period of 15 years and can be downloaded from:

https://www.kaggle.com/therohk/million-headlines/data

You can also find this file in the lab6 folder.

We read the CSV file using the pandas framework.

In [2]:
import pandas as pd

#### Adapt the path below to point to your local copy of the data set
data = pd.read_csv('abcnews-date-text.csv', on_bad_lines="warn");
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

Let's have a look at the data:

In [6]:
print(len(documents))
print(documents[:5])

1244184
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


We are going to use the *gensim* package to build our LDA models from the data.
Before building the model, we are going to preprocess the texts.

## Data Pre-processing
We will perform the following steps:

* Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All stopwords are removed.
* Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are stemmed — words are reduced to their root form.

In order to apply these processing steps, we first load the gensim and nltk libraries

In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

C:\Users\szcze\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\szcze\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\szcze\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\szcze\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
def lemmatize_stemming(text):
    return lemmatizer.lemmatize(text)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
           # result.append(token)
            result.append(lemmatize_stemming(token))
    return result

In [14]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepayer', 'group', 'want', 'compulsory', 'local', 'govt', 'voting']


We now apply the preprocessing to all the headlines and print the first 10 results

In [16]:
processed_docs = documents['headline_text'].map(preprocess)
### print the first 10 results
processed_docs[:10]

0          [decides, community, broadcasting, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4              [strike, affect, australian, traveller]
5               [ambitious, olsson, win, triple, jump]
6          [antic, delighted, record, breaking, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                       [australia, locked, timetable]
Name: headline_text, dtype: object

## Bag of Words on the Data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.
We are going to use the *Dictionary* function to derive a dictionary with counts from the headlines.

In [19]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcasting
1 community
2 decides
3 licence
4 aware
5 defamation
6 witness
7 call
8 infrastructure
9 protection
10 summit


## Gensim filter_extremes
Filter out tokens that appear in
less than 15 documents (absolute number) or
more than 0.5 documents (fraction of total corpus size, not absolute number).
after the above two steps, keep only the first 100000 most frequent tokens.

In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## Gensim doc2bow
For each document we create a dictionary reporting how many words and how many times those words appear. 
Gensim provides the *doc2bow* function to create a BoW vector representation for a document.
Save this to ‘bow_corpus’, then check our selected document earlier.

In [23]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(164, 1), (241, 1), (615, 1), (891, 1), (4175, 1), (4176, 1), (4177, 1)]

Preview Bag Of Words for our sample preprocessed document.

In [25]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 164 ("govt") appears 1 time.
Word 241 ("group") appears 1 time.
Word 615 ("local") appears 1 time.
Word 891 ("want") appears 1 time.
Word 4175 ("compulsory") appears 1 time.
Word 4176 ("ratepayer") appears 1 time.
Word 4177 ("voting") appears 1 time.


## TF-IDF
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.


In [31]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.6161125947380649),
 (1, 0.3308772069039591),
 (2, 0.5681053683635203),
 (3, 0.43379930266554434)]


## Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’. This takes a while.
Look at the documentation of *gensim* for further details:

https://radimrehurek.com/gensim/models/ldamulticore.html

As parameters, we pass the corpus data as BoW (a list of lists of tuples), the prefixed number of topics, the actual words and the number of passes and workers used for modeling.

In [34]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [36]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.036*"case" + 0.025*"court" + 0.022*"police" + 0.021*"woman" + 0.020*"child" + 0.020*"vaccine" + 0.018*"murder" + 0.014*"death" + 0.014*"charged" + 0.014*"face"
Topic: 1 
Words: 0.039*"police" + 0.023*"school" + 0.016*"house" + 0.014*"missing" + 0.012*"guilty" + 0.011*"drum" + 0.011*"family" + 0.011*"crash" + 0.010*"melbourne" + 0.010*"search"
Topic: 2 
Words: 0.016*"life" + 0.015*"premier" + 0.013*"time" + 0.013*"claim" + 0.012*"speaks" + 0.012*"pandemic" + 0.012*"hotel" + 0.011*"road" + 0.011*"say" + 0.011*"young"
Topic: 3 
Words: 0.022*"government" + 0.020*"coast" + 0.017*"national" + 0.015*"plan" + 0.015*"live" + 0.015*"federal" + 0.012*"gold" + 0.012*"care" + 0.012*"farmer" + 0.010*"aged"
Topic: 4 
Words: 0.040*"coronavirus" + 0.024*"covid" + 0.020*"open" + 0.017*"lockdown" + 0.016*"hospital" + 0.016*"australian" + 0.015*"dy" + 0.015*"final" + 0.014*"sydney" + 0.012*"tasmanian"
Topic: 5 
Words: 0.027*"election" + 0.022*"health" + 0.019*"say" + 0.017*"people" + 0.

Can you distinguish different topics using the words in each topic and their corresponding weights?

## Running LDA using TF-IDF

In [39]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.012*"government" + 0.009*"country" + 0.009*"coronavirus" + 0.008*"health" + 0.008*"federal" + 0.007*"hour" + 0.006*"election" + 0.006*"andrew" + 0.006*"funding" + 0.005*"covid"
Topic: 1 Word: 0.014*"royal" + 0.011*"commission" + 0.010*"story" + 0.010*"christmas" + 0.009*"young" + 0.008*"daniel" + 0.007*"testing" + 0.006*"mount" + 0.005*"win" + 0.005*"allegedly"
Topic: 2 Word: 0.008*"peter" + 0.006*"grandstand" + 0.006*"june" + 0.006*"social" + 0.006*"prime" + 0.006*"know" + 0.006*"territory" + 0.005*"america" + 0.005*"hong" + 0.005*"minister"
Topic: 3 Word: 0.016*"coronavirus" + 0.016*"covid" + 0.011*"case" + 0.010*"record" + 0.010*"market" + 0.009*"price" + 0.008*"rise" + 0.008*"rate" + 0.007*"australian" + 0.007*"share"
Topic: 4 Word: 0.018*"donald" + 0.013*"drum" + 0.008*"tuesday" + 0.008*"wednesday" + 0.008*"michael" + 0.008*"hill" + 0.007*"sport" + 0.007*"friday" + 0.007*"league" + 0.006*"outback"
Topic: 5 Word: 0.024*"news" + 0.016*"interview" + 0.013*"scott" + 0

Again, can you distinguish different topics using the words in each topic and their corresponding weights? Do you observe any differences with the BoW version? Do these differences make sense given the information value weighing by the *tfidf* method?

## Performance evaluation by classifying sample document using LDA Bag of Words model
We will check where our test document would be classified.

In [42]:
processed_docs[4310]

['ratepayer', 'group', 'want', 'compulsory', 'local', 'govt', 'voting']

Document 4310 is already represented in the correct way. We can directly pass it to our *lda_model* to get the similarity scores for each topic. We represent each topic by printing 

In [44]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3982229232788086	 
Topic: 0.022*"government" + 0.020*"coast" + 0.017*"national" + 0.015*"plan" + 0.015*"live" + 0.015*"federal" + 0.012*"gold" + 0.012*"care" + 0.012*"farmer" + 0.010*"aged"

Score: 0.2394091933965683	 
Topic: 0.027*"election" + 0.022*"health" + 0.019*"say" + 0.017*"people" + 0.016*"minister" + 0.015*"change" + 0.014*"business" + 0.014*"morrison" + 0.012*"labor" + 0.011*"andrew"

Score: 0.150007426738739	 
Topic: 0.040*"covid" + 0.036*"trump" + 0.035*"queensland" + 0.029*"victoria" + 0.024*"coronavirus" + 0.023*"australia" + 0.020*"record" + 0.019*"news" + 0.015*"market" + 0.014*"south"

Score: 0.13729117810726166	 
Topic: 0.039*"police" + 0.023*"school" + 0.016*"house" + 0.014*"missing" + 0.012*"guilty" + 0.011*"drum" + 0.011*"family" + 0.011*"crash" + 0.010*"melbourne" + 0.010*"search"

Score: 0.012511841021478176	 
Topic: 0.020*"border" + 0.017*"north" + 0.015*"china" + 0.014*"protest" + 0.013*"west" + 0.012*"amid" + 0.011*"say" + 0.011*"president" + 0.010*

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

### Analyzing our LDA model

Now that we have a trained model let’s visualize the topics for interpretability. 
To do so, we’ll use a popular visualization package, *pyLDAvis* which is designed to help interactively with:

1. Better understanding and interpreting individual topics, and
2. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most frequent and/or “relevant” terms, using different values of the λ parameter. This can help when you’re trying to assign a human interpretable name or “meaning” to each topic.
For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

You need to install *pyldavis* through the command line, following the instructions:

https://anaconda.org/conda-forge/pyldavis

WARNING: running the next cell takes a long time and you need some memory to run it. However, the result is spectacular.

In [47]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, corpus=bow_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

## Some other useful functions

In [49]:
#get the top 20 words and their weights for a specific topic
topic_id=1
top_terms=20
for wordid, score in lda_model.get_topic_terms(topic_id, top_terms):
    print(wordid, ":", dictionary[wordid], ":", score)

237 : police : 0.038638264
1116 : school : 0.023448538
284 : house : 0.015517109
313 : missing : 0.013521588
829 : guilty : 0.012267797
3276 : drum : 0.011006853
805 : family : 0.0108161485
195 : crash : 0.010679763
899 : melbourne : 0.010291002
541 : search : 0.010139787
1023 : announces : 0.010085628
649 : dead : 0.009929524
111 : help : 0.009835574
361 : northern : 0.009358252
746 : emergency : 0.009136911
545 : finance : 0.008877363
9637 : daniel : 0.008723972
286 : white : 0.007905053
1072 : fatal : 0.0074055726
593 : teen : 0.007297871


In [50]:
#### Utility function to get the id for a word

def get_id_for_word(dictionary, word):
    for k, v in dictionary.iteritems():
        if (v==word):
            return k
    return -1

In [51]:
top_terms=20
index=get_id_for_word(dictionary,'market')
for topic_id, score in lda_model.get_term_topics(index):
    print("Topic:", topic_id)
    for wordid, score in lda_model.get_topic_terms(topic_id, top_terms):
        print(wordid, ":", dictionary[wordid], ":", score)


Topic: 6
18896 : covid : 0.04016209
8173 : trump : 0.035681576
2232 : queensland : 0.034556262
1619 : victoria : 0.029464388
18895 : coronavirus : 0.024344463
37 : australia : 0.02273457
26 : record : 0.019539807
1363 : news : 0.019030537
1188 : market : 0.0154747665
144 : south : 0.014171698
620 : victorian : 0.009559413
16 : australian : 0.009398527
3206 : street : 0.009261894
1422 : interview : 0.00879436
679 : brisbane : 0.007967626
856 : india : 0.0074655465
1416 : warning : 0.0072388407
1415 : travel : 0.007150871
2264 : wall : 0.00696839
3056 : video : 0.0069476203


## Saving and loading your model for re-use

Building a model takes time.Once you have a stable model, you can save it to disk and reload it later.

In [57]:
# Save model to disk.
temp_file = "./model"
lda_model.save(temp_file)

# Load a potentially pretrained model from disk.
loaded_lda = gensim.models.LdaModel.load(temp_file)

## Testing model on unseen document

In [59]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'

In order to compare any new text against the topic model, we first need to process it in the same way as we processed the input texts for the model.
We apply the same preprocessing function and next apply the *doc2bow* function to represent it using the same vector representation as we used for modeling.

In [62]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
print(bow_vector)


[(888, 1), (1088, 1), (1922, 1), (5668, 1), (12411, 1)]


We can now pass this representation of the unseen document into the model to compare it against all the topics.
The next function returns in index to the topics and a similarity score for the new document. We print the scores and the topics with the top 5 words.

In [64]:
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic_id {}\t Topic: {}".format(score, index, lda_model.print_topic(index, 5)))

Score: 0.34985679388046265	 Topic_id 8	 Topic: 0.054*"australia" + 0.024*"donald" + 0.018*"restriction" + 0.013*"world" + 0.012*"australian"
Score: 0.32158583402633667	 Topic_id 6	 Topic: 0.040*"covid" + 0.036*"trump" + 0.035*"queensland" + 0.029*"victoria" + 0.024*"coronavirus"
Score: 0.21184629201889038	 Topic_id 9	 Topic: 0.020*"border" + 0.017*"north" + 0.015*"china" + 0.014*"protest" + 0.013*"west"
Score: 0.016674112528562546	 Topic_id 3	 Topic: 0.022*"government" + 0.020*"coast" + 0.017*"national" + 0.015*"plan" + 0.015*"live"
Score: 0.01667405664920807	 Topic_id 5	 Topic: 0.027*"election" + 0.022*"health" + 0.019*"say" + 0.017*"people" + 0.016*"minister"
Score: 0.016673319041728973	 Topic_id 0	 Topic: 0.036*"case" + 0.025*"court" + 0.022*"police" + 0.021*"woman" + 0.020*"child"
Score: 0.016672400757670403	 Topic_id 1	 Topic: 0.039*"police" + 0.023*"school" + 0.016*"house" + 0.014*"missing" + 0.012*"guilty"
Score: 0.016672400757670403	 Topic_id 2	 Topic: 0.016*"life" + 0.015*"pre

This text matches best with topic 5 although the score is not very high!

### Updating the model with a new document

We can also use the unseen documents to extend our model and update the topics. This is useful when processing texts in a stream.

In [71]:
# Update the model by incrementally training on the new corpus.

other_texts = [['computer', 'time', 'graph'],['survey', 'response', 'eps'],['human', 'system', 'computer']]
other_corpus = [dictionary.doc2bow(text) for text in other_texts]

# Update the model by incrementally training on the new corpus.
lda_model.update(other_corpus)  # update the LDA model with additional documents



C:\Users\szcze\anaconda3\Lib\site-packages\gensim\models\ldamodel.py:850: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


## End of this notebook